# 1. Para desarrollar este desafío necesitarás los siguientes archivos:
● incidents.pkl
● officers.pkl
● subjects.pkl

a. Carga los datos y crea un DataFrame con cada uno de ellos.






In [3]:
# Importar pandas
import pandas as pd


a. Carga los datos y crea un DataFrame con cada uno de ellos.

In [5]:


incidents = pd.read_pickle("incidents.pkl")
officers = pd.read_pickle("officers.pkl")
subjects = pd.read_pickle("subjects.pkl")

# Visualizar la forma de los datos
print(incidents.shape, officers.shape, subjects.shape)

(219, 15) (370, 6) (223, 6)


In [6]:
incidents.columns

Index(['case_number', 'date', 'location', 'subject_statuses', 'subject_weapon',
       'subjects', 'subject_count', 'officers', 'officer_count',
       'grand_jury_disposition', 'attorney_general_forms_url', 'summary_url',
       'summary_text', 'latitude', 'longitude'],
      dtype='object')

In [7]:
officers.columns


Index(['case_number', 'race', 'gender', 'last_name', 'first_name',
       'full_name'],
      dtype='object')

In [8]:
subjects.columns


Index(['case_number', 'race', 'gender', 'last_name', 'first_name',
       'full_name'],
      dtype='object')

In [9]:
#Identificamos la comuna en comun con todos los dataframes para unirlas
common_columns = set(officers.columns) & set(subjects.columns) & set(incidents.columns) #Interseccion de nombres de columnas en un set
print("La columna comun es:", common_columns)

La columna comun es: {'case_number'}


b. Genera una tabla que contenga la unión de las 3 tablas. hint: utiliza sufijos para  las
columnas que se llaman igual usando el parámetro suffixes de pd.merge().

In [11]:

# Unir incidents con officers. Incidents son eventos unicos, se mantienen por si solos
df_union1 = pd.merge(incidents, officers, on='case_number', suffixes=('', '_officer')) 
#Registros de officers no quedan etiquetados porque no se repiten nombres de columnas con incidents



In [12]:
# Unir con subjects
df_union = pd.merge(df_union1, subjects, on='case_number', suffixes=('', '_subject')) #Registros de subjets quedan etiquetados

# Visualizar la forma de los datos
print(df_union.shape)

(377, 25)


In [13]:
df_union.columns

Index(['case_number', 'date', 'location', 'subject_statuses', 'subject_weapon',
       'subjects', 'subject_count', 'officers', 'officer_count',
       'grand_jury_disposition', 'attorney_general_forms_url', 'summary_url',
       'summary_text', 'latitude', 'longitude', 'race', 'gender', 'last_name',
       'first_name', 'full_name', 'race_subject', 'gender_subject',
       'last_name_subject', 'first_name_subject', 'full_name_subject'],
      dtype='object')

c. Verifica si hay filas duplicadas; si es así, elimínalas.

In [15]:

duplicates = df_union.duplicated().sum()
print(f"Número de filas duplicadas: {duplicates}")



Número de filas duplicadas: 0


In [16]:
registros_por_caso = df_union['case_number'].value_counts()
print(registros_por_caso)

# Filtrar los case_number con más de un registro
registros_repetidos = registros_por_caso[registros_por_caso > 1]

# Imprimir solo esos
print(registros_repetidos)


case_number
134472-2015    15
380792W        13
165193-2016    12
199250-2015     9
94073V          8
               ..
161582R         1
379145V         1
636873T         1
523445N         1
327705X         1
Name: count, Length: 216, dtype: int64
case_number
134472-2015    15
380792W        13
165193-2016    12
199250-2015     9
94073V          8
               ..
862009M         2
165748Y         2
989995N         2
559487M         2
309130W         2
Name: count, Length: 72, dtype: int64


No existen registros duplicados propiamente tal, pero existen 72 casos que poseen mas de un registro debido a que tienen mas de un sujeto o mas de un oficial involucrado

d. ¿Cuántos sujetos de género F hay en el DataFrame resultante? hint: usa el método

In [19]:

genero_subject = df_union['gender_subject'].value_counts()
print(genero_subject)

print("Los sujetos F (femenino) son:", genero_subject['F'])

gender_subject
M    368
F      9
Name: count, dtype: int64
Los sujetos F (femenino) son: 9


e. ¿En cuántos números de caso hay por lo menos una sospechosa que sea mujer? hint:
utiliza el método unique() para obtener los valores únicos de una columna específica de
un DataFrame luego de filtrar.

In [21]:
# Filtrar por género femenino y obtener casos únicos
casos_sujeto_f = df_union[df_union['gender_subject'] == 'F']

n_casos_sujeto_f = casos_sujeto_f['case_number'].nunique()
print("Número de casos con al menos una mujer sospechosa:", n_casos_sujeto_f)


Número de casos con al menos una mujer sospechosa: 7


f. Genera una tabla pivote que muestre en las filas el género del oficial y en las columnas el
género del subject. ¿Cómo interpretas los valores que muestra esta vista?

In [23]:
tabla_pivote = pd.pivot_table(df_union,
                             index='gender',
                             columns='gender_subject',
                             values='case_number',
                             aggfunc='count',
                             fill_value=0)

print(tabla_pivote)


gender_subject  F    M
gender                
F               2   18
M               7  350


Es mucho mas común que los oficiales masculinos esten a cargo de procesar casos con sujetos masculinos

# 2. Para continuar con el desarrollo de este desafío, necesitarás el archivos Cleaned_DS_Jobs.csv


a. Carga los datos y crea un DataFrame con ellos.

In [27]:
import numpy as np

df = pd.read_csv("Cleaned_DS_Jobs.csv")

In [28]:
df['Salary Estimate']



0      137-171 
1      137-171 
2      137-171 
3      137-171 
4      137-171 
         ...   
655    105-167 
656    105-167 
657    105-167 
658    105-167 
659    105-167 
Name: Salary Estimate, Length: 660, dtype: object

b. Utiliza la siguiente lista de valores que serán considerados como nulos:
["na", "NA", -1, "0", "-1", "null", "n/a", "N/A", "NULL"]
(hint: utiliza el método replace para reemplazar los valores indicados por np.nan)


In [30]:
# Lista de valores nulos
valores_nulos = ["na", "NA", -1, "0", "-1", "null", "n/a", "N/A", "NULL"]

# Reemplazar con np.nan
df.replace(valores_nulos, np.nan, inplace=True)


c. Elimina todas las filas con datos faltantes. (hint: utiliza el método .dropna())

In [32]:
df.dropna(inplace=True)


d. A partir de la columna “Salary Estimate”, genera dos columnas: Salario Estimado Mínimo
y Máximo. (hint: Utiliza el método apply sobre la columna.)

In [34]:
# Crear dos columnas de salario maximo y minimo a partir del salario estimado
df[["Salario Estimado Mínimo", "Salario Estimado Máximo"]] = df["Salary Estimate"].apply(lambda x: pd.Series(x.split("-")))

print(df["Salario Estimado Máximo"].head(5))
print(df["Salario Estimado Mínimo"].head(5))

0     171 
32    131 
38    131 
45    131 
54    131 
Name: Salario Estimado Máximo, dtype: object
0     137
32     75
38     75
45     75
54     75
Name: Salario Estimado Mínimo, dtype: object


e. Realiza la recodificación de la columna Size con los valores de la siguiente tabla: (hint:
utilice reemplazo con diccionario usando el método replace sobre la columna.)


In [36]:
# Diccionario de recodificación
recodificacion = {
    "10000+ employees": "Mega Empresas",
    "5001 to 10000 employees": "Grandes Empresas",
    "1001 to 5000 employees": "Medianas Empresas",
    "201 to 500 employees": "Pequeñas Empresas",
    "51 to 200 employees": "Pequeñas Grandes Empresas",
    "501 to 1000 employees": "Microempresas",
    "Unknown": "Empresas sin Información"
}

# Guardar valores en una nueva columna recodificada
df["Size Descripción"] = df["Size"].replace(recodificacion)



f. Finalmente, genera una tabla pivote que muestre la media del salario estimado mínimo y
la media del salario estimado máximo por tamaño de empresa. (hint: utiliza
pd.pivot_table para generar la vista adecuada con las columnas generadas.)


In [38]:
# Transformar objetos a valor numerico
df["Salario Estimado Mínimo"] = pd.to_numeric(df["Salario Estimado Mínimo"])
df["Salario Estimado Máximo"] = pd.to_numeric(df["Salario Estimado Máximo"])

#Crear tabla pivote con tamaño empresas como filas y salarios estimados como columnas. Agregar por la media
pivot = pd.pivot_table(
    df,
    index='Size Descripción',  
    values=['Salario Estimado Mínimo', 'Salario Estimado Máximo'], 
    aggfunc='mean'  
)

print(pivot)


                           Salario Estimado Máximo  Salario Estimado Mínimo
Size Descripción                                                           
Empresas sin Información                110.500000                73.000000
Grandes Empresas                        138.875000                92.125000
Medianas Empresas                       137.461538                93.923077
Mega Empresas                           151.111111                97.888889
Microempresas                           146.235294               100.176471
Pequeñas Empresas                       141.142857                93.571429
Pequeñas Grandes Empresas               137.666667               100.666667
